In [ ]:
# watsonx AI 서비스 테스트 노트북
import requests
import base64
import json
from pathlib import Path

# ===============================
# 1. 기본 설정 및 토큰 발급
# ===============================

# IBM Cloud API 키 설정 (여기에 실제 API 키 입력)
API_KEY = "fnuO5k8mU3q_NeAw7e2EzYg4URqqup2pdbnENGjItcj8"  # 🔧 실제 API 키로 변경하세요

def get_watson_token(api_key):
    """IBM Watson 토큰 발급"""
    try:
        token_response = requests.post(
            'https://iam.cloud.ibm.com/identity/token', 
            data={
                "apikey": api_key, 
                "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
            }
        )
        
        if token_response.status_code == 200:
            mltoken = token_response.json()["access_token"]
            print("✅ 토큰 발급 성공!")
            return mltoken
        else:
            print(f"❌ 토큰 발급 실패: {token_response.status_code}")
            print(token_response.text)
            return None
            
    except Exception as e:
        print(f"❌ 토큰 발급 에러: {str(e)}")
        return None

# 토큰 발급 테스트
print("🔑 토큰 발급 테스트...")
token = get_watson_token(API_KEY)


In [ ]:
# ===============================
# 2. 텍스트 전용 테스트
# ===============================

def test_text_only(token, user_message):
    """텍스트 전용 watsonx AI 서비스 테스트"""
    
    if not token:
        print("❌ 토큰이 없습니다!")
        return None
    
    headers = {
        'Content-Type': 'application/json', 
        'Authorization': f'Bearer {token}'
    }
    
    # watsonx AI 서비스 URL
    ai_service_url = 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/c59e817c-448f-45f1-bc34-df12f190ac0d/ai_service?version=2021-05-01'
    
    # 페이로드
    payload = {
        "messages": [
            {
                "content": user_message,
                "role": "user"
            }
        ]
    }
    
    print(f"📤 전송 메시지: {user_message}")
    print(f"📤 페이로드: {json.dumps(payload, indent=2, ensure_ascii=False)}")
    
    try:
        response = requests.post(
            ai_service_url,
            json=payload,
            headers=headers
        )
        
        print(f"📊 응답 상태: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"📥 응답 전체: {json.dumps(result, indent=2, ensure_ascii=False)}")
            
            # 다양한 응답 형식 처리
            if 'choices' in result and len(result['choices']) > 0:
                answer = result['choices'][0]['message']['content']
                print(f"✅ 답변: {answer}")
                return answer
            elif 'generated_text' in result:
                answer = result['generated_text']
                print(f"✅ 답변: {answer}")
                return answer
            else:
                print("🤔 응답 형식을 확인할 수 없습니다.")
                return result
        else:
            print(f"❌ 요청 실패: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 에러 발생: {str(e)}")
        return None

# 텍스트 테스트 실행
print("\n" + "="*50)
print("📝 텍스트 전용 테스트")
print("="*50)

test_messages = [
    "안녕하세요!",
    "아스피린의 효능을 설명해주세요.",
    "당뇨병 환자가 주의해야 할 약물은 무엇인가요?"
]

for msg in test_messages:
    print(f"\n🧪 테스트: {msg}")
    print("-" * 30)
    test_text_only(token, msg)


In [ ]:
# ===============================
# 3. 이미지 처리 테스트
# ===============================

def get_mime_type(file_path):
    """파일 확장자를 기반으로 MIME 타입 반환"""
    ext = Path(file_path).suffix.lower()
    mime_types = {
        '.jpg': 'image/jpeg', '.jpeg': 'image/jpeg',
        '.png': 'image/png', '.gif': 'image/gif',
        '.bmp': 'image/bmp', '.tiff': 'image/tiff',
        '.webp': 'image/webp'
    }
    return mime_types.get(ext, 'image/jpeg')

def test_image_vision(token, image_path, user_question):
    """이미지 + 텍스트 watsonx vision 모델 테스트"""
    
    if not token:
        print("❌ 토큰이 없습니다!")
        return None
    
    # 이미지 파일 존재 확인
    if not Path(image_path).exists():
        print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")
        return None
    
    try:
        # 이미지를 base64로 인코딩
        with open(image_path, "rb") as f:
            img_data = f.read()
            img64 = base64.b64encode(img_data).decode('utf-8')
        
        mime_type = get_mime_type(image_path)
        print(f"📷 이미지 로드: {image_path} ({mime_type})")
        print(f"📏 이미지 크기: {len(img_data)} bytes")
        
        headers = {
            'Content-Type': 'application/json', 
            'Authorization': f'Bearer {token}'
        }
        
        # watsonx AI 서비스 URL
        ai_service_url = 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/c59e817c-448f-45f1-bc34-df12f190ac0d/ai_service?version=2021-05-01'
        
        # vision 모델용 페이로드
        payload = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_question},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:{mime_type};base64,{img64}"
                            }
                        }
                    ]
                }
            ]
        }
        
        print(f"📤 질문: {user_question}")
        print(f"📤 페이로드 구조 확인 (이미지 제외): ")
        payload_debug = payload.copy()
        payload_debug["messages"][0]["content"][1]["image_url"]["url"] = f"data:{mime_type};base64,[BASE64_DATA...]"
        print(json.dumps(payload_debug, indent=2, ensure_ascii=False))
        
        response = requests.post(
            ai_service_url,
            json=payload,
            headers=headers
        )
        
        print(f"📊 응답 상태: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print(f"📥 응답 전체: {json.dumps(result, indent=2, ensure_ascii=False)}")
            
            # 다양한 응답 형식 처리
            if 'choices' in result and len(result['choices']) > 0:
                answer = result['choices'][0]['message']['content']
                print(f"✅ 이미지 분석 결과: {answer}")
                return answer
            elif 'generated_text' in result:
                answer = result['generated_text']
                print(f"✅ 이미지 분석 결과: {answer}")
                return answer
            else:
                print("🤔 응답 형식을 확인할 수 없습니다.")
                return result
        else:
            print(f"❌ 요청 실패: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 이미지 처리 에러: {str(e)}")
        return None

# 이미지 테스트 실행
print("\n" + "="*50)
print("🖼️ 이미지 처리 테스트")
print("="*50)

# 🔧 실제 이미지 파일 경로로 변경하세요
test_image_path = "test_image.jpg"  # 여기에 실제 이미지 경로 입력

test_questions = [
    "이 이미지가 무엇인지 설명해주세요.",
    "이 의료 문서에서 약물 정보를 추출해주세요.",
    "이 처방전의 주요 내용을 요약해주세요."
]

for question in test_questions:
    print(f"\n🧪 이미지 테스트: {question}")
    print("-" * 40)
    test_image_vision(token, test_image_path, question)


In [4]:
# ===============================
# 🔧 watsonx Vision 디버깅 코드
# ===============================

import requests
import base64
import json
from pathlib import Path
import hashlib

# 새로운 deployment URL 사용
AI_SERVICE_URL = "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/c59e817c-448f-45f1-bc34-df12f190ac0d/ai_service?version=2021-05-01"

def debug_base64_encoding(image_path):
    """Base64 인코딩 디버깅"""
    print("🔍 Base64 인코딩 디버깅")
    print("-" * 40)
    
    try:
        with open(image_path, "rb") as f:
            img_data = f.read()
            img64 = base64.b64encode(img_data).decode('utf-8')
        
        print(f"📏 원본 이미지 크기: {len(img_data)} bytes")
        print(f"📏 Base64 문자열 길이: {len(img64)} characters")
        print(f"🔢 Base64 앞 50글자: {img64[:50]}...")
        print(f"🔢 Base64 뒤 50글자: ...{img64[-50:]}")
        
        # Base64 패딩 확인
        padding_needed = 4 - (len(img64) % 4)
        if padding_needed != 4:
            print(f"⚠️ Base64 패딩 필요: {padding_needed} 개의 '=' 추가 필요")
            img64_padded = img64 + ('=' * padding_needed)
            print(f"✅ 패딩 추가된 길이: {len(img64_padded)}")
            return img64_padded
        else:
            print("✅ Base64 패딩 정상")
            return img64
            
    except Exception as e:
        print(f"❌ Base64 인코딩 에러: {str(e)}")
        return None

def test_different_image_formats(token, image_path, question):
    """다양한 이미지 형식으로 테스트"""
    
    print("\n🖼️ 이미지 형식별 테스트")
    print("=" * 50)
    
    # Base64 디버깅
    img64 = debug_base64_encoding(image_path)
    if not img64:
        return
    
    # 다양한 MIME 타입 테스트
    mime_types_to_test = [
        "image/jpeg",
        "image/jpg", 
        "image/png",
        "image/gif",
        "image/webp"
    ]
    
    headers = {
        'Content-Type': 'application/json', 
        'Authorization': f'Bearer {token}'
    }
    
    for mime_type in mime_types_to_test:
        print(f"\n🧪 MIME 타입 테스트: {mime_type}")
        print("-" * 30)
        
        # 방법 1: image_url 형식
        payload_v1 = {
            "messages": [
                {
                    "role": "user", 
                    "content": [
                        {"type": "text", "text": question},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:{mime_type};base64,{img64}"
                            }
                        }
                    ]
                }
            ]
        }
        
        print("📤 image_url 형식 테스트...")
        response = requests.post(AI_SERVICE_URL, json=payload_v1, headers=headers)
        print(f"📊 상태: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ 성공!")
            result = response.json()
            print(f"📥 응답: {json.dumps(result, indent=2, ensure_ascii=False)}")
            return result
        else:
            print(f"❌ 실패: {response.text[:200]}...")

def test_simplified_vision_formats(token, image_path, question):
    """간단한 vision 형식들 테스트"""
    
    print("\n🔍 간단한 Vision 형식 테스트")
    print("=" * 50)
    
    img64 = debug_base64_encoding(image_path)
    if not img64:
        return
    
    headers = {
        'Content-Type': 'application/json', 
        'Authorization': f'Bearer {token}'
    }
    
    # 방법 1: 직접 base64 필드
    test_formats = [
        {
            "name": "직접 base64 필드",
            "payload": {
                "messages": [
                    {
                        "role": "user",
                        "content": question,
                        "image": img64
                    }
                ]
            }
        },
        {
            "name": "images 배열",
            "payload": {
                "messages": [
                    {
                        "role": "user", 
                        "content": question
                    }
                ],
                "images": [img64]
            }
        },
        {
            "name": "attachments 형식",
            "payload": {
                "messages": [
                    {
                        "role": "user",
                        "content": question,
                        "attachments": [
                            {
                                "type": "image",
                                "data": img64
                            }
                        ]
                    }
                ]
            }
        },
        {
            "name": "단순 content 배열",
            "payload": {
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {"text": question},
                            {"image": img64}
                        ]
                    }
                ]
            }
        }
    ]
    
    for test_format in test_formats:
        print(f"\n🧪 {test_format['name']} 테스트")
        print("-" * 30)
        
        try:
            response = requests.post(AI_SERVICE_URL, json=test_format["payload"], headers=headers)
            print(f"📊 상태: {response.status_code}")
            
            if response.status_code == 200:
                print("✅ 성공!")
                result = response.json()
                print(f"📥 응답: {json.dumps(result, indent=2, ensure_ascii=False)}")
                return result
            else:
                error_text = response.text
                if len(error_text) > 300:
                    error_text = error_text[:300] + "..."
                print(f"❌ 실패: {error_text}")
                
        except Exception as e:
            print(f"❌ 에러: {str(e)}")

def check_deployment_capabilities(token):
    """배포된 모델의 capability 확인"""
    
    print("\n🔎 모델 Capability 확인")
    print("=" * 50)
    
    headers = {
        'Content-Type': 'application/json', 
        'Authorization': f'Bearer {token}'
    }
    
    # 간단한 텍스트만으로 모델 정보 요청
    test_queries = [
        "당신이 어떤 모델인지 소개해주세요.",
        "이미지를 처리할 수 있나요?",
        "지원하는 기능을 알려주세요.",
        "vision capability가 있나요?"
    ]
    
    for query in test_queries:
        print(f"\n🧪 질문: {query}")
        print("-" * 30)
        
        payload = {
            "messages": [
                {
                    "role": "user",
                    "content": query
                }
            ]
        }
        
        try:
            response = requests.post(AI_SERVICE_URL, json=payload, headers=headers)
            print(f"📊 상태: {response.status_code}")
            
            if response.status_code == 200:
                result = response.json()
                if 'choices' in result and len(result['choices']) > 0:
                    answer = result['choices'][0]['message']['content']
                    print(f"✅ 답변: {answer}")
                elif 'generated_text' in result:
                    print(f"✅ 답변: {result['generated_text']}")
                else:
                    print(f"🤔 응답 형식: {json.dumps(result, indent=2, ensure_ascii=False)}")
            else:
                print(f"❌ 실패: {response.text[:200]}...")
                
        except Exception as e:
            print(f"❌ 에러: {str(e)}")

def test_image_preprocessing(image_path):
    """이미지 전처리 테스트"""
    
    print("\n🖼️ 이미지 전처리 테스트") 
    print("=" * 50)
    
    try:
        from PIL import Image
        import io
        
        # 원본 이미지 정보
        with Image.open(image_path) as img:
            print(f"📏 원본 크기: {img.size}")
            print(f"📄 원본 형식: {img.format}")
            print(f"🎨 원본 모드: {img.mode}")
            
            # RGB 변환 (RGBA나 다른 모드일 경우)
            if img.mode != 'RGB':
                print(f"🔄 {img.mode} → RGB 변환")
                img = img.convert('RGB')
            
            # 크기 조정 (너무 큰 경우)
            max_size = 1024
            if max(img.size) > max_size:
                print(f"📐 크기 조정: {max_size}px 이하로")
                img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
                print(f"📏 조정된 크기: {img.size}")
            
            # JPEG로 저장하여 base64 인코딩
            buffer = io.BytesIO()
            img.save(buffer, format='JPEG', quality=85)
            img_data = buffer.getvalue()
            img64 = base64.b64encode(img_data).decode('utf-8')
            
            print(f"📏 처리된 이미지 크기: {len(img_data)} bytes")
            print(f"📏 Base64 길이: {len(img64)} characters")
            
            return img64
            
    except ImportError:
        print("❌ PIL(Pillow) 라이브러리가 필요합니다: pip install Pillow")
        return None
    except Exception as e:
        print(f"❌ 이미지 전처리 에러: {str(e)}")
        return None


In [5]:
# ===============================
# 🚀 종합 디버깅 실행
# ===============================

def comprehensive_vision_debug(token, image_path, question="이 이미지를 분석해주세요."):
    """종합 vision 디버깅"""
    
    print("🔧 watsonx Vision 종합 디버깅 시작")
    print("=" * 60)
    
    # 1. 모델 capability 확인
    check_deployment_capabilities(token)
    
    # 2. 이미지 전처리 테스트  
    preprocessed_img64 = test_image_preprocessing(image_path)
    
    # 3. 다양한 형식 테스트
    test_different_image_formats(token, image_path, question)
    
    # 4. 간단한 형식 테스트
    test_simplified_vision_formats(token, image_path, question)
    
    # 5. 전처리된 이미지로 재시도
    if preprocessed_img64:
        print("\n🔄 전처리된 이미지로 재시도")
        print("=" * 50)
        
        headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
        
        payload = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": question},
                        {
                            "type": "image_url", 
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{preprocessed_img64}"
                            }
                        }
                    ]
                }
            ]
        }
        
        response = requests.post(AI_SERVICE_URL, json=payload, headers=headers)
        print(f"📊 상태: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 전처리 후 성공!")
            print(f"📥 응답: {json.dumps(result, indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ 전처리 후에도 실패: {response.text[:300]}...")

# 실행
if token:
    comprehensive_vision_debug(token, "test_image.jpg")
else:
    print("❌ 토큰을 먼저 발급받으세요!")


🔧 watsonx Vision 종합 디버깅 시작

🔎 모델 Capability 확인

🧪 질문: 당신이 어떤 모델인지 소개해주세요.
------------------------------
📊 상태: 200
✅ 답변: 안녕하세요! 저는 닥터왓슨입니다. 전문 약사로서, 사용자의 질문에 친절하게 답해드릴 수 있도록 최선을 다하겠습니다. 약학에 관련된 질문이나 상담이 필요하시면 언제든지 물어봐주세요!

🧪 질문: 이미지를 처리할 수 있나요?
------------------------------
📊 상태: 200
✅ 답변: 네, 이미지 처리가 가능합니다.

🧪 질문: 지원하는 기능을 알려주세요.
------------------------------
📊 상태: 200
✅ 답변: 안녕하세요! 닥터왓슨입니다. 전문 약사로서, 사용자의 질문에 친절하게 답해드리겠습니다. 지원하는 기능은 다음과 같습니다.

1. **질문에 대한 답변**: 사용자의 질문에 대한 답변을 제공합니다. 질문의 범위는 의학, 약학, 건강, 질병, 약물, 치료법 등입니다.
2. **의학 정보 제공**: 의학 관련 정보를 제공합니다. 예를 들어, 질병의 원인, 증상, 치료법, 예방 방법 등입니다.
3. **약물 정보 제공**: 약물 관련 정보를 제공합니다. 예를 들어, 약물의 성분, 작용 기전, 사용 방법, 부작용 등입니다.
4. **건강 정보 제공**: 건강 관련 정보를 제공합니다. 예를 들어, 건강 유지 방법, 질병 예방 방법, 건강检查 등입니다.
5. **의학 용어 설명**: 의학 용어를 설명합니다. 예를 들어, 의학 용어의 정의, 설명 등입니다.

이러한 기능을 통해 사용자의 질문에 친절하게 답해드리겠습니다. 질문이 있으신가요?

🧪 질문: vision capability가 있나요?
------------------------------
📊 상태: 200
✅ 답변: Vision capability is the ability for a model to "see" and understand image

In [6]:
# ===============================
# 📋 최종 결론 체크
# ===============================

def final_vision_conclusion(token):
    """Vision 지원 여부 최종 결론"""
    
    print("\n🎯 최종 결론")
    print("=" * 50)
    
    # 모델에게 직접 물어보기
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
    
    questions = [
        "You are deployed as what model? Please tell me your exact model name and capabilities.",
        "Can you process images? Do you have vision capabilities?",
        "What types of input do you support? Text only or multimodal?",
        "If I send you a base64 encoded image, can you analyze it?"
    ]
    
    for q in questions:
        payload = {"messages": [{"role": "user", "content": q}]}
        response = requests.post(AI_SERVICE_URL, json=payload, headers=headers)
        
        if response.status_code == 200:
            result = response.json()
            if 'choices' in result and len(result['choices']) > 0:
                answer = result['choices'][0]['message']['content']
                print(f"❓ {q}")
                print(f"✅ {answer}")
                print("-" * 50)

# 최종 결론 실행
if token:
    final_vision_conclusion(token)



🎯 최종 결론
❓ You are deployed as what model? Please tell me your exact model name and capabilities.
✅ I couldn't find the model name for 닥터왓슨. Let me try again.
--------------------------------------------------
❓ Can you process images? Do you have vision capabilities?
✅ No, I don't have vision capabilities. I can process text-based input and output, but I cannot process images or any visual data. I'm a large language model, my primary function is to understand and generate human-like text, but I don't have the capability to see or interpret visual information.
--------------------------------------------------
❓ What types of input do you support? Text only or multimodal?
✅ I support text-only input. I'm a large language model, I don't have the capability to process or understand images, videos, or audio files. I can only respond to text-based input, so please feel free to ask me any questions or provide information in text format, and I'll do my best to assist you!
-------------------